In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('dataset_37_diabetes.csv')
df.head()

In [ ]:
df = df.rename(columns={"class' { tested_negative": 'result'})

df.to_csv('dataset_37_diabetes.csv', index=False)

In [ ]:
df.duplicated().sum()

In [ ]:
df["result"].value_counts()

In [ ]:
## balancing the data
Negative = df[df["result"] == 'tested_negative']
Positive = df[df["result"] == 'tested_positive']

In [ ]:
## Check data is balance or not

fig = plt.figure(figsize=(8,5))
df.result.value_counts().plot(kind="bar")
plt.show()

In [ ]:
Negative = Negative.sample(250)
Negative.shape

In [ ]:
balance_df = pd.concat([Negative,Positive],axis=0)
balance_df.head()

In [ ]:
## Check data is balance or not

fig = plt.figure(figsize=(8,5))
balance_df.result.value_counts().plot(kind="bar")
plt.show()

In [ ]:
cat_col = balance_df.drop(["Unnamed: 9"],axis=1)
cat_col.head()

In [ ]:
cat_col =cat_col.select_dtypes(include="O")
cat_col.head()

In [ ]:
num_col = balance_df.drop(["Unnamed: 9"],axis=1)
num_col.head()

In [ ]:
num_col = num_col.select_dtypes(exclude="O")
num_col.head()

In [ ]:
pd.get_dummies(cat_col).astype(int).head()

In [ ]:
cat_col["result"].value_counts()

In [ ]:
dt = {
    'tested_positive':1,
    'tested_negative':0
}

cat_col["result"] = cat_col["result"].map(dt)
cat_col["result"].head()

In [ ]:
final_df = pd.concat([cat_col,num_col],axis=1)
final_df.head()

In [ ]:
## feature selection

x = final_df.drop("result",axis=1)
y = final_df[["result"]]

In [ ]:
## training and testing classification

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(
    x,y,test_size=0.2,random_state=42
)

In [ ]:
# encoding of y_test

if len(y_test.shape) > 1 and y_test.shape[1] > 1:
    y_test = np.argmax(y_test, axis=1)

In [ ]:
y_test.shape

In [ ]:
## Algorithm

from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(x_train,y_train)

In [ ]:
# to save the model
import joblib,pickle
joblib.dump(model,"diabities.lb")

In [ ]:
model.score(x_test,y_test)

In [ ]:
model.score(x_train,y_train)

In [ ]:
pred = model.predict(x_test)
y_test["prediction"] = pred
y_test

In [ ]:
## Model evaluation

from sklearn.metrics import confusion_matrix,classification_report

In [ ]:
confusion_matrix(y_test["result"],y_test["prediction"])

In [ ]:
sns.heatmap(confusion_matrix(y_test["result"],y_test["prediction"]),annot=True,fmt='d')

In [ ]:
## classification report
print(classification_report(y_test["result"],y_test["prediction"]))

In [ ]:
# ## ROC curve

# from sklearn.metrics import roc_curve,auc
# # Compute ROC curve and AUC
# fpr, tpr, thresholds = roc_curve(y_test, pred)
# roc_auc = auc(fpr, tpr)

# # Plot ROC curve
# plt.figure(figsize=(8,6))
# plt.plot(fpr, tpr, color='blue', lw=2, label='Logistic Regression (AUC = %0.2f)' % roc_auc)
# plt.plot([0, 1], [0, 1], color='red', linestyle='--')  # Random guess line
# plt.xlim([0.0, 1.0])
# plt.ylim([0.0, 1.05])
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('ROC Curve - Logistic Regression')
# plt.legend(loc="lower right")
# plt.grid(True)
# plt.show()